In [3]:
from google.colab import files

# Upload locally downloaded file 'credit'
uploaded = files.upload()

Saving credit_sample.csv to credit_sample.csv


In [2]:
!pip install sdv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 12.6 MB/s 
     |████████████████████████████████| 139 kB 72.4 MB/s 
     |████████████████████████████████| 47 kB 5.2 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 1.6 MB 66.0 MB/s 
     |████████████████████████████████| 61 kB 446 kB/s 
     |████████████████████████████████| 9.4 MB 76.4 MB/s 
     |████████████████████████████████| 965 kB 70.8 MB/s 
     |████████████████████████████████| 295 kB 46.6 MB/s 
     |████████████████████████████████| 662 kB 71.3 MB/s 
     |████████████████████████████████| 280 kB 77.4 MB/s 
     |████████████████████████████████| 15.3 MB 68.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
  Attempting uninstall: psutil
    Found existing installation: 

In [3]:
import pandas as pd
import numpy as np
import time
from matplotlib import pyplot as plt

from sdv.tabular import TVAE
from sdv.tabular import CTGAN
from sdv.evaluation import evaluate

import seaborn as sns
from sdv.metrics.tabular import NumericalRadiusNearestNeighbor as NRNN

import warnings
warnings.filterwarnings('ignore')

In [4]:
batch_sizes = [300]
epochs = [200]

gan_models = []
vae_models = []
#copula_models = []

MODEL_TVAE = "TVAE"
MODEL_CTGAN = "CTGAN"

In [5]:
#metrics

#ml_metrics = ['MulticlassEfficacyMetric', "MulticlassDecisionTreeClassifier", "MulticlassMLPClassifier"]

statistical_metrics = ['CSTest', 'KSComplement', 'ContinuousKLDivergence', 'DiscreteKLDivergence']
privacy_metrics = ["NumericalRadiusNearestNeighbor"]

all_metrics = statistical_metrics + privacy_metrics

synthetic_data_mapping = {}
key_list = []

In [6]:
df = pd.read_csv("credit_sample.csv")
df.name = "credit"
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42721 entries, 0 to 42720
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    42721 non-null  float64
 1   V1      42721 non-null  float64
 2   V2      42721 non-null  float64
 3   V3      42721 non-null  float64
 4   V4      42721 non-null  float64
 5   V5      42721 non-null  float64
 6   V6      42721 non-null  float64
 7   V7      42721 non-null  float64
 8   V8      42721 non-null  float64
 9   V9      42721 non-null  float64
 10  V10     42721 non-null  float64
 11  V11     42721 non-null  float64
 12  V12     42721 non-null  float64
 13  V13     42721 non-null  float64
 14  V14     42721 non-null  float64
 15  V15     42721 non-null  float64
 16  V16     42721 non-null  float64
 17  V17     42721 non-null  float64
 18  V18     42721 non-null  float64
 19  V19     42721 non-null  float64
 20  V20     42721 non-null  float64
 21  V21     42721 non-null  float64
 22

In [ ]:
#df['Class'].value_counts()

In [ ]:
#(df['Class'].value_counts()) / len(df) * 100

In [ ]:
#df_sample = df.groupby('Class', group_keys=False).apply(lambda x: x.sample(frac=0.15))
#df_sample.name = "credit_sample"

In [ ]:
#df_sample['Class'].value_counts()

In [ ]:
#(df_sample['Class'].value_counts()) / len(df_sample) * 100

In [7]:
def get_key(batch, epoch, model):
    return model + ': Batch size - ' + str(batch)+ ", Epochs - "+ str(epoch)

In [8]:
for batch in batch_sizes:
    for epoch in epochs:
        gan_models.append(CTGAN(batch_size=batch, verbose=True, epochs=epoch))
        vae_models.append(TVAE(batch_size=batch, epochs=epoch))
        key_list.append(get_key(batch=batch, epoch=epoch, model=MODEL_TVAE))
        key_list.append(get_key(batch=batch, epoch=epoch, model=MODEL_CTGAN))
        synthetic_data_mapping[get_key(batch=batch, epoch=epoch, model=MODEL_TVAE)] = pd.DataFrame()
        synthetic_data_mapping[get_key(batch=batch, epoch=epoch, model=MODEL_CTGAN)] = pd.DataFrame()

In [9]:
def evaluate_model(model, key, metrics, df):
    
    print('Training in Progress - ' + key + '\n')
    
    # Record training time
    start = time.time()
    model.fit(df)
    end = time.time()
    
    print( '\n' + model.__class__.__name__ + ' trained. \nTraining time: ' + str(end-start) + ' seconds \n')
    syn_data = model.sample(len(df))
    #syn_data.name = df.name + '-' + model.__class__.__name__
    syn_data.name = key

    # Save Generated Synthetic Data for each model in a dictionary 
    synthetic_data_mapping[key] = syn_data   
    
    # Record evaluation time
    start = time.time()
    ee = evaluate(syn_data, df, metrics=metrics , aggregate=False)
    end = time.time()
    print("Synthetic Data Evaluation - " + key + '\n')
    display(ee)
    print('\nEvaluation time: ' + str(end-start) + ' seconds \n')
    
    
    # Save the model
    #saved_model_name = model.__class__.__name__ + '_' + df.name + '.pkl'
    #model.save(saved_model_name)
    #saved_models[saved_model_name] = model

In [10]:
key_list

['TVAE: Batch size - 300, Epochs - 200',
 'CTGAN: Batch size - 300, Epochs - 200']

In [11]:
i=0
j=0
for key in key_list:
    if MODEL_TVAE in key:
        evaluate_model(model= vae_models[i],
        key=key,
        metrics=all_metrics,
        df= df)
        i = i + 1
    else:
        evaluate_model(model= gan_models[j],
        key=key,
        metrics=all_metrics,
        df= df)
        j = j+1


Training in Progress - TVAE: Batch size - 300, Epochs - 200


TVAE trained. 
Training time: 988.4401693344116 seconds 

Synthetic Data Evaluation - TVAE: Batch size - 300, Epochs - 200



,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,CSTest,Chi-Squared,NaN,NaN,0.0,1.0,MAXIMIZE,"Cannot find fields of types ('boolean', 'categ..."
1,KSComplement,Inverted Kolmogorov-Smirnov D statistic,0.691853,0.691853,0.0,1.0,MAXIMIZE,None
2,ContinuousKLDivergence,Continuous Kullback–Leibler Divergence,0.842969,0.842969,0.0,1.0,MAXIMIZE,None
3,DiscreteKLDivergence,Discrete Kullback–Leibler Divergence,NaN,NaN,0.0,1.0,MAXIMIZE,"Cannot find fields of types ('boolean', 'categ..."
4,NumericalRadiusNearestNeighbor,Numerical Radius Nearest Neighbor,NaN,NaN,0.0,inf,MAXIMIZE,`key_fields` must be passed either directly or...



Evaluation time: 4.3087756633758545 seconds 

Training in Progress - CTGAN: Batch size - 300, Epochs - 200

Epoch 1, Loss G: -1.4255,Loss D:  0.5305
Epoch 2, Loss G: -1.9619,Loss D: -0.1183
Epoch 3, Loss G: -2.3285,Loss D: -0.0066
Epoch 4, Loss G: -1.9647,Loss D:  0.0254
Epoch 5, Loss G: -2.1823,Loss D: -0.2859
Epoch 6, Loss G: -2.6974,Loss D:  0.3426
Epoch 7, Loss G: -2.8675,Loss D:  0.1544
Epoch 8, Loss G: -3.3150,Loss D:  0.4430
Epoch 9, Loss G: -3.5825,Loss D:  0.1731
Epoch 10, Loss G: -3.7361,Loss D:  0.1595
Epoch 11, Loss G: -3.4884,Loss D: -0.0430
Epoch 12, Loss G: -3.8660,Loss D: -0.0580
Epoch 13, Loss G: -3.8156,Loss D: -0.1850
Epoch 14, Loss G: -3.9481,Loss D:  0.3588
Epoch 15, Loss G: -4.1005,Loss D:  0.0763
Epoch 16, Loss G: -3.7792,Loss D:  0.2849
Epoch 17, Loss G: -4.0068,Loss D:  0.3694
Epoch 18, Loss G: -3.8907,Loss D:  0.0915
Epoch 19, Loss G: -3.7395,Loss D: -0.1446
Epoch 20, Loss G: -3.4757,Loss D:  0.2426
Epoch 21, Loss G: -3.6083,Loss D:  0.0155
Epoch 22, Loss G: 

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,CSTest,Chi-Squared,NaN,NaN,0.0,1.0,MAXIMIZE,"Cannot find fields of types ('boolean', 'categ..."
1,KSComplement,Inverted Kolmogorov-Smirnov D statistic,0.659705,0.659705,0.0,1.0,MAXIMIZE,None
2,ContinuousKLDivergence,Continuous Kullback–Leibler Divergence,0.640157,0.640157,0.0,1.0,MAXIMIZE,None
3,DiscreteKLDivergence,Discrete Kullback–Leibler Divergence,NaN,NaN,0.0,1.0,MAXIMIZE,"Cannot find fields of types ('boolean', 'categ..."
4,NumericalRadiusNearestNeighbor,Numerical Radius Nearest Neighbor,NaN,NaN,0.0,inf,MAXIMIZE,`key_fields` must be passed either directly or...



Evaluation time: 4.237553596496582 seconds 



In [12]:
def generate_corr_heatmap(real, syn_vae, syn_gan, title):
        
    # Correlation Matrix for each feature
    fig, ax = plt.subplots(1, 3, figsize=(18,6))
    corrmat = round(real.corr(), 2)
    top_corr_features = corrmat.index
    #plot heat map
    sns.heatmap(real[top_corr_features].corr(), ax=ax[0], annot=False,fmt='.2f',cmap="BuGn").set(title="Real")

    corrmat = round(syn_vae.corr(), 2)
    top_corr_features = corrmat.index
    #plot heat map
    sns.heatmap(syn_vae[top_corr_features].corr(), ax=ax[1], annot=False,fmt='.2f',cmap="BuGn").set(title="TVAE")

    corrmat = round(syn_gan.corr(), 2)
    top_corr_features = corrmat.index
    #plot heat map
    sns.heatmap(syn_gan[top_corr_features].corr(), ax=ax[2], annot=False,fmt='.2f',cmap="BuGn").set(title="CTGAN")

    fig.suptitle(title)
    return fig

In [13]:
def get_plot_params(n_plots):
        n_cols = 3
        n_rows = int(n_plots/n_cols) if (n_plots%n_cols == 0) else int(n_plots/n_cols + 1)
        return n_rows, n_cols

def generate_plots_numerical(real, syn_vae, syn_gan):
    num_cols = [col for col in real.columns 
                if np.real_if_close.dtypes[col] in ['float64', 'int64']]
    
    n_rows, n_cols = get_plot_params(len(num_cols))
    
    fig, ax = plt.subplots(n_rows, n_cols, figsize=(7*n_cols,7*n_rows))
    i,j = 0, 0
    for col in num_cols:
        sns.kdeplot(data=real, ax=ax[i][j], x=col, label="Real")
        sns.kdeplot(data=syn_vae, ax=ax[i][j], x=col, label="TVAE")
        sns.kdeplot(data=syn_gan, ax=ax[i][j], x=col, label="CTGAN")
        ax[i][j].legend()
        j = (j+1)%3
        if(j==0): i += 1

In [2]:
# Generate report
i=1
for key in key_list:

  if(not synthetic_data_mapping[key].empty):
    # Plot univariate and bivariate correlations
    g = sns.PairGrid(synthetic_data_mapping[key], hue="Class")
    g.map_diag(sns.kdeplot)
    g.map_offdiag(sns.scatterplot)
    g.add_legend()
    g.fig.suptitle('Synthetic Data Distribution: ' + key)

    if i%2==0:
      # Plot heatmap
      generate_corr_heatmap(df, synthetic_data_mapping[key_list[key_list.index[key]-1]], title=key)

      generate_plots_numerical(df, synthetic_data_mapping[key], title=key)

    i = i+1



NameError: ignored